In [67]:
import numpy as np
import pandas as pd

import os # used for navigating to image path
import imageio # used for writing images

from sklearn.preprocessing import LabelEncoder

import keras

#  Keras preprocessing
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image_dataset_from_directory

# Keras modeling
from keras.models import Sequential
from keras.layers import  Lambda , Dense, Flatten, Dropout, Conv2D, MaxPool2D
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

In [74]:
train = pd.read_csv('train.csv', error_bad_lines=False, nrows = 300, sep = '\t')
test = pd.read_csv('test.csv', error_bad_lines=False, sep = '\t')
train_text = train['productname']
test_text = test['productname']
train_tags = train['label']
test_tags = test['label']

In [69]:
train.head()

,imageid,label,productname
0,2653,Bags,Murcia Women Leather Office Grey Bag
1,55997,Others,Colorbar Velvet Matte Temptation Lipstick 24MA
2,2640,Shoes,Carlton London Men Brown Formal Shoes
3,40565,Topwear,W Women Maroon Kurta
4,38932,Bottomwear,Gini and Jony Girls Pink Leggings


In [70]:
epochs = 3
top_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

In [71]:
from keras.preprocessing import text, sequence

max_words = 10000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_text)
x_train = tokenize.texts_to_matrix(train_text)
test_text = test_text.astype(str)
x_test = tokenize.texts_to_matrix(test_text)

In [75]:
from keras import utils


encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)


In [82]:
top_model = Sequential()
top_model.add(layers.Input(shape=(max_words,)))
top_model.add(Flatten())
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.1))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(13, activation='sigmoid'))


In [103]:
top_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               2560256   
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_14 (Dense)             (None, 13)                3341      
Total params: 2,629,389
Trainable params: 2,629,389
Non-trainable params: 0
_________________________________________________________________


In [85]:
low_model = Sequential()
low_model.add(layers.Input(shape=(200,)))
low_model.add(Flatten())
low_model.add(Dense(256, activation='relu'))
low_model.add(Dropout(0.1))
low_model.add(Dense(256, activation='relu'))
low_model.add(Dense(13, activation='sigmoid'))


In [86]:
from keras.models import Model
from keras.layers import *

mergedOut = Add()([top_model.output,low_model.output])
newModel = Model([top_model.input,low_model.input], mergedOut)

In [102]:
newModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
history1 = newModel.fit([x_train, pd.DataFrame(x_train).iloc[:,:200] ], y_train, batch_size=batch_size, epochs=5, verbose=1, validation_split=0.1)
score1 = newModel.evaluate([x_test, pd.DataFrame(x_test).iloc[:,:200] ], y_test, batch_size=batch_size, verbose=1)

Epoch 1/5
9/9 [==============================] - 5s 196ms/step - loss: 1.2864 - accuracy: 0.3660 - val_loss: 1.2330 - val_accuracy: 0.4000
Epoch 2/5
9/9 [==============================] - 1s 69ms/step - loss: 1.0907 - accuracy: 0.3594 - val_loss: 1.1351 - val_accuracy: 0.4000
Epoch 3/5
9/9 [==============================] - 0s 58ms/step - loss: 0.9014 - accuracy: 0.4399 - val_loss: 0.9873 - val_accuracy: 0.5667
Epoch 4/5
9/9 [==============================] - 0s 58ms/step - loss: 0.6542 - accuracy: 0.7755 - val_loss: 0.7787 - val_accuracy: 0.7667
Epoch 5/5
125/125 [==============================] - 2s 11ms/step - loss: 0.6853 - accuracy: 0.8145


In [80]:
top_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
history = top_model.fit(x_train, y_train, batch_size=batch_size, epochs=5, verbose=1, validation_split=0.1)
score = top_model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

Epoch 1/5
9/9 [==============================] - 3s 144ms/step - loss: 2.4955 - accuracy: 0.3130 - val_loss: 2.2116 - val_accuracy: 0.4000
Epoch 2/5
9/9 [==============================] - 0s 51ms/step - loss: 2.0993 - accuracy: 0.3607 - val_loss: 1.6503 - val_accuracy: 0.4000
Epoch 3/5
9/9 [==============================] - 0s 49ms/step - loss: 1.6313 - accuracy: 0.3688 - val_loss: 1.3967 - val_accuracy: 0.4333
Epoch 4/5
9/9 [==============================] - 0s 29ms/step - loss: 1.2316 - accuracy: 0.5655 - val_loss: 1.2199 - val_accuracy: 0.6333
Epoch 5/5
125/125 [==============================] - 2s 14ms/step - loss: 1.1187 - accuracy: 0.7135 0s - loss: 1.1115 - accuracy


In [ ]:
#https://www.tensorflow.org/guide/keras/save_and_serialize
model.save("my_model")

In [ ]:
)